In [1]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
from mtranslate import translate
import http.cookies
import re
import json

In [2]:
cookies = pd.read_json('cookies.json')

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.63',}

In [4]:
sess = requests.Session()

In [5]:
for index, cookie in cookies.iterrows():
    sess.cookies.set(cookie['name'], cookie['value'])

In [6]:
site = sess.get('https://s.taobao.com/search?spm=a21wu.241046-global.6977698868.164.4103b6cbbgbVCY&q=%E9%A5%B0%E5%93%81DIY&acm=lb-zebra-241046-2075394.1003.4.1812236&scm=1003.4.lb-zebra-241046-2075394.OTHER_15206383875223_1812236',
                   headers=headers,)

In [7]:
soup = BeautifulSoup(site.content)

In [8]:
def get_items(res):
    g_page_config = re.search(r"g_page_config = (.*?);\n", res.text)
    page_config_json = json.loads(g_page_config.group(1))
    page_items = page_config_json['mods']['itemlist']['data']['auctions']
    result = []
    for each_item in page_items:
        dict1 = dict.fromkeys(('nid','title','detail_url','view_price','view_sales','nick'))
        dict1['nid'] = each_item['nid']
        dict1['title'] = each_item['title']
        dict1['detail_url'] = "https:"+each_item['detail_url']
        dict1['view_price'] = each_item['view_price']
        dict1['view_sales'] = each_item['view_sales']
        dict1['nick'] = each_item['nick']
        result.append(dict1)
    return result

In [18]:
get_items(site)[1]['detail_url']

'https://detail.tmall.com/item.htm?id=624327944034&ad_id=&am_id=&cm_id=140105335569ed55e27b&pm_id=&abbucket=5#detail'

In [59]:
site1 = sess.get('https://detail.tmall.com/item.htm?id=631135729332&ad_id=&am_id=&cm_id=140105335569ed55e27b&pm_id=&abbucket=5#detail')

In [60]:
soup = BeautifulSoup(site1.content)

In [61]:
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="webkit" name="renderer"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<script>
    (function(w,d){
    try{
        var l,url='//mdskip.taobao.com/core/initItemDetail.htm?isUseInventoryCenter=false&cartEnable=true&service3C=false&isApparel=false&isSecKill=false&tmallBuySupport=true&isAreaSell=false&tryBeforeBuy=false&offlineShop=false&itemId=631135729332&showShopProm=false&isPurchaseMallPage=false&itemGmtModified=1617400089000&isRegionLevel=false&household=false&sellerPreview=false&queryMemberRight=true&addressLevel=2&isForbidBuyItem=false',isg=document.cookie.match('(^|;) ?l=([^;]*)(;|$)'),isg2 =document.cookie.match('(^|;) ?isg=([^;]*)(;|$)');
        if(!url){return;}
        var arr=["callback=setMdskip","timestamp="+(+new Date()),"isg="+(isg&&isg[2]),"isg2="+(isg2&&isg2[2])],reg=/[?&^](ip|campaignId|key|abt|cat_id|q|u_channel|areaId|sdShopId)=([^&]+)/g,params=w.location.search;
        while(r=reg.exec

In [62]:
soup.find('textarea', {'class':'ks-datalazyload'})

<textarea class="ks-datalazyload">
                <div class="bd">
            <div class="shop-rate" data-spm="d4918101">
                                    <h4>店铺动态评分<!--盟主aaa-->
                        <span class="compare">与同行业相比</span>
                    </h4>
                    <ul>
                        <li>
                            描述相符：<a href="//rate.taobao.com/user-rate-UvCIYMGHLOm8uvNTT.htm" target="_blank">
                                                            <em class="count" title="4.8分">4.8</em>
                                                    <span class="rateinfo" title="计算规则:(同行业平均分-店铺得分)/(同行业平均分-同行业店铺最低得分)">
                                                                                                                    <b class="lower"></b><em class="lower">0.59%</em>
                                                                                                            </span>
                                                    </a>
      